### 保存载入体系介绍

#### 基础API保存载入体系

>训练场景：使用paddle.save/load
>
>推理部署：使用paddle.jie.save/load(动态图)；paddle.satic.save/load_inference_model
>

#### 高层API保存载入体系

>paddle.Model.fit(训练接口，同事带有保存参数功能)
>
>paddle.Model.save
>
>paddle.Model.load

### 训练调优场景的（模型、参数）保存与载入

>训练阶段主要保存模型的参数，当模型的训练时间很长时，为了防止意外中断需要保存模型训练的参数
>

#### 动态图参数保存载入

>使用 paddle.save/load 结合Layer和Optimizer的state_dict达成目的，此处的state_dict是对象的持久化参数的载体，dict的key为参数名，value为参数真实值

In [1]:
import numpy as np
import pandas as pd
import paddle
import paddle.nn as nn
import paddle.optimizer as opt
import warnings
warnings.filterwarnings("ignore")

In [2]:
BATCH_SIZE = 16
BATCH_NUM = 4
EPOCH_NUM = 4

IMAGE_SIZE = 784
CLASS_NUM = 10

In [3]:
# 定义一个随机数据集 
class RandomDateset(paddle.io.Dataset):
    def __init__(self,num_samples):
        self.num_samples = num_samples
        
    def __getitem__(self,idx):
        image = np.random.random([IMAGE_SIZE]).astype('float32')
        label = np.random.randint(0,CLASS_NUM-1,(1,)).astype('int64')
        return image,label
    
    def __len__(self):
        return self.num_samples

In [4]:
# 子类组网
class LinearNet(nn.Layer):
    def __init__(self):
        super(LinearNet,self).__init__()
        self._linear = nn.Linear(IMAGE_SIZE,CLASS_NUM)
        
    def forward(self,x):
        return self._linear(x)


In [8]:
# 定义训练函数
def train(layer,loader,loass_fn,opt):
    for epoch_id in range(EPOCH_NUM):
        for batch_id,data in enumerate(loader()):
            x = data[0]
            y = data[1]
            predicts = layer(x)
            loss = loss_fn(predicts,y)
            acc = paddle.metric.accuracy(predicts,y)
            loss.backward()
            opt.step()
            opt.clear_grad()
            print(f"epoch_id:{epoch_id},batch_id:{batch_id},loss:{loss.numpy()},acc:{acc.numpy()}")

In [9]:
# 定义网络、损失函数，优化器，精确度
layer = LinearNet()
loss_fn = paddle.nn.CrossEntropyLoss()
adam = paddle.optimizer.Adam(learning_rate=0.01,parameters=layer.parameters())
# 创建数据加载器，开始训练
dataset = RandomDateset(BATCH_NUM*BATCH_SIZE)
loader = paddle.io.DataLoader(dataset,batch_size=BATCH_SIZE,shuffle=True,drop_last=True,num_workers=2)



In [10]:
train(layer,loader,loss_fn,adam)

epoch_id:0,batch_id:0,loss:[3.058033],acc:[0.0625]
epoch_id:0,batch_id:1,loss:[6.3019342],acc:[0.125]
epoch_id:0,batch_id:2,loss:[6.0211387],acc:[0.0625]
epoch_id:0,batch_id:3,loss:[4.9375143],acc:[0.0625]
epoch_id:1,batch_id:0,loss:[6.2604437],acc:[0.0625]
epoch_id:1,batch_id:1,loss:[5.0825515],acc:[0.125]
epoch_id:1,batch_id:2,loss:[3.6923733],acc:[0.125]
epoch_id:1,batch_id:3,loss:[3.1607432],acc:[0.0625]
epoch_id:2,batch_id:0,loss:[3.6203542],acc:[0.0625]
epoch_id:2,batch_id:1,loss:[3.6932602],acc:[0.0625]
epoch_id:2,batch_id:2,loss:[3.9843092],acc:[0.]
epoch_id:2,batch_id:3,loss:[4.0350485],acc:[0.0625]
epoch_id:3,batch_id:0,loss:[3.6471248],acc:[0.1875]
epoch_id:3,batch_id:1,loss:[3.4326372],acc:[0.]
epoch_id:3,batch_id:2,loss:[3.4537778],acc:[0.125]
epoch_id:3,batch_id:3,loss:[3.7570505],acc:[0.125]


In [11]:
# 保存参数
paddle.save(layer.state_dict(),"linear_net.pdparams")
paddle.save(adam.state_dict(),"adam.pdopt")

In [12]:
# 载入参数
layer_state_dict = paddle.load("linear_net.pdparams")
opt_state_dict = paddle.load("adam.pdopt")

In [13]:
# 使用下面方法将参数更新到模型中
layer.set_state_dict(layer_state_dict)
adam.set_state_dict(opt_state_dict)